In [1]:
from ultralytics import YOLO
import motmetrics as mm
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np
import cv2
import os
import shutil
import torch
import sys
from time import time
from datetime import timedelta
from PIL import Image
import json

In [2]:
PATH_VIDEO = 'C:\\Users\\Freo\\Desktop\\kaggle\\test_task\\renew\\data\\videos\\31-03-2024-09%3A34%3A24.mp4'
OUTPUT_VIDEO = 'output_tracked_video.mp4'

path_to_gt = 'C:\\Users\\Freo\\Desktop\\kaggle\\test_task\\renew\\data\\datasets\\mot_dataset\\gt\\gt.txt'  
path_to_tracker_results = 'C:\\Users\\Freo\\Desktop\\kaggle\\test_task\\renew\\mot_results.txt'



path_to_custom_track = 'C:\\Users\\Freo\\Desktop\\kaggle\\test_task\\renew\\data\\models\\ultralytics\\my_track.yaml'

number_frames = 9000

In [3]:
PATH_MODEL = "C:\\Users\\Freo\\Desktop\\kaggle\\test_task\\renew\\data\\models\\ultralytics\\yolov10x_v2_4_best.pt"

# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

# Загрузка модели на GPU
model = YOLO(PATH_MODEL, verbose=False).to(device)

Using device: cuda


In [4]:
def initialize_video(PATH_VIDEO, OUTPUT_VIDEO):
    cap = cv2.VideoCapture(PATH_VIDEO)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (frame_width, frame_height))
    return cap, out, fps, frame_width, frame_height

In [5]:
def process_frame(frame, model, track_history):
    frame_np = frame.copy()  # Сохраняем оригинальный numpy array для трекинга
    frame = frame.transpose(2, 0, 1)  # Перестановка осей в порядок CHW
    frame = np.expand_dims(frame, axis=0)  # Добавляем измерение для batch

    frame = torch.from_numpy(frame).float().div(255.0).cuda(non_blocking=True) if device == 'cuda' else torch.from_numpy(frame).float().div(255.0).cpu()

    # Трекинг
    results = model.track(frame_np, persist=True, tracker=path_to_custom_track)
    
    # Проверка наличия идентификаторов треков
    track_ids = results[0].boxes.id.cpu().numpy().astype(int) if results[0].boxes.id is not None else []

    # Получение координат боксов
    boxes = results[0].boxes.xywh.cpu() if results[0].boxes.xywh is not None else []

    # Аннотируем кадр (отрисовываем трекинг)
    annotated_frame = results[0].plot()

    # Сохранение трека для каждого объекта
    for box, track_id in zip(boxes, track_ids):
        x, y, w, h = box
        track = track_history[track_id]
        track.append((float(x), float(y)))  # x, y - центр
        if len(track) > 90:
            track.pop(0)
        
        # Рисуем линии трекинга
        points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
        cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=2)

    return annotated_frame, track_ids, boxes

In [6]:
cap, out, fps, frame_width, frame_height = initialize_video(PATH_VIDEO, OUTPUT_VIDEO)

# История трекинга
track_history = defaultdict(lambda: [])

# Ограничение на количество кадров для отладки
max_frames = number_frames

# Инициализация счетчика кадров
frame_count = 0

# Подготовка к записи в MOT формат
mot_results = []

# Старт времени после инициализации
start_time = time()

# Обработка видео
while cap.isOpened() and frame_count < max_frames:
    success, frame = cap.read()

    if not success:
        break

    # Приводим кадр к размеру, соответствующему предыдущему кадру
    frame = cv2.resize(frame, (frame_width, frame_height))  # Изменение размера до исходного

    # Обработка кадра
    annotated_frame, track_ids, boxes = process_frame(frame, model, track_history)

    # Запись кадра в выходное видео
    out.write(annotated_frame)

    # Запись данных в формате MOT
    for box, track_id in zip(boxes, track_ids):
        x, y, w, h = box
        
        # Преобразование центра в координаты верхнего левого угла
        x_min = x - w / 2
        y_min = y - h / 2
        
        mot_results.append((frame_count + 1, track_id, x_min, y_min, w, h, 1, 2, 1.0))

    if device == 'cuda':
        # Очистка кэша GPU с синхронизацией
        torch.cuda.synchronize()
        torch.cuda.empty_cache()



    # Увеличение счетчика обработанных кадров
    frame_count += 1

# Остановка времени сразу после последней операции с кадрами
stop_time = time()

# Завершение работы с видео
if out:
    out.release()
cap.release()

# Вычисление времени обработки
total_time = stop_time - start_time
avg_time_per_frame = total_time / frame_count if frame_count > 0 else 0
print(f'''\nСтатистика обработки:
Количество обработанных кадров: {frame_count}
Общее время обработки: {total_time:.2f} секунд
Среднее время обработки одного кадра: {avg_time_per_frame * 1000:.2f} мс

Видео с трекингом сохранено в {OUTPUT_VIDEO}
Результаты трекинга сохранены в mot_results.txt
''')

# Сохранение результатов в файл MOT
with open('mot_results.txt', 'w') as f:
    for result in mot_results:
        f.write(f"{result[0]},{result[1]},{result[2]},{result[3]},{result[4]},{result[5]},{result[6]},{result[7]},{result[8]}\n")


0: 480x640 1 PET (transparent) (brown), 2 PET (transparent)s, 49.4ms
Speed: 4.0ms preprocess, 49.4ms inference, 23.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 PET (transparent) (brown), 2 PET (transparent)s, 40.5ms
Speed: 3.0ms preprocess, 40.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 PET (transparent) (brown), 2 PET (transparent)s, 41.5ms
Speed: 2.6ms preprocess, 41.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 PET (transparent) (brown), 2 PET (transparent)s, 37.1ms
Speed: 2.0ms preprocess, 37.1ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 PET (transparent) (brown), 2 PET (transparent)s, 35.1ms
Speed: 2.0ms preprocess, 35.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 PET (transparent) (brown), 2 PET (transparent)s, 35.8ms
Speed: 2.4ms preprocess, 35.8ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 64

In [7]:
# Функция для загрузки данных из файла в формат motmetrics
def load_mot_results(path, max_frames=100):
    data = {}
    with open(path, 'r') as f:
        for line in f.readlines():
            frame_id, track_id, x, y, w, h, _, _, _ = map(float, line.strip().split(','))
            frame_id = int(frame_id)
            if frame_id > max_frames:
                break
            if frame_id not in data:
                data[frame_id] = []
            # Преобразование координат в формат (ID, x_center, y_center, width, height)
            data[frame_id].append((track_id, x + w / 2, y + h / 2, w, h))
    return data

# Загрузка данных из файлов
gt_data = load_mot_results(path_to_gt, max_frames=number_frames)
tracker_data = load_mot_results(path_to_tracker_results, max_frames=number_frames)

# Инициализация аккумулятора
acc = mm.MOTAccumulator(auto_id=True)

# Сопоставление GT и результатов трекинга по кадрам
for frame_id in range(1, 9001):  # Первые 100 кадров
    gt_frame = gt_data.get(frame_id, [])
    tracker_frame = tracker_data.get(frame_id, [])
    
    # Разделяем данные по идентификаторам и координатам
    gt_ids, gt_points = zip(*[(d[0], (d[1], d[2])) for d in gt_frame]) if gt_frame else ([], [])
    tracker_ids, tracker_points = zip(*[(d[0], (d[1], d[2])) for d in tracker_frame]) if tracker_frame else ([], [])

    # Вычисляем расстояния между треками и GT 
    distances = mm.distances.norm2squared_matrix(gt_points, tracker_points, max_d2=10000)
    
    # Обновляем аккумулятор с текущими данными
    acc.update(gt_ids, tracker_ids, distances)

# Вычисление метрик
mh = mm.metrics.create()
summary = mh.compute(acc, metrics=['mota', 'motp', 'idf1', 'precision', 'recall'], name='acc')

In [8]:
summary

,mota,motp,idf1,precision,recall
acc,0.923127,233.051184,0.93893,0.996594,0.930562


In [9]:
print(f'''avg per frame {avg_time_per_frame * 1000:.2f}
frame total {frame_count}
''')

avg per frame 126.86
frame total 9000

